## Visualise stokes cartesian model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

pass

[Lyrebird.local:01988] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.Lyrebird.501/jf.0/2849308672/sm_segment.Lyrebird.501.a9d50000.0 could be created.


In [3]:
ls -trl output/cartesian* 

output/cartesian:
total 0
drwxr-xr-x@  294 lmoresi  staff    9408 Aug 26 21:59 Ra1e3.0/
drwxr-xr-x@  909 lmoresi  staff   29088 Aug 27 16:34 Ra1e4.0/
drwxr-xr-x@ 2009 lmoresi  staff   64288 Aug 27 18:41 Ra1e5.0/
drwxr-xr-x  7517 lmoresi  staff  240544 Aug 28 21:19 Ra1e7.0/
drwxr-xr-x  6758 lmoresi  staff  216256 Aug 29 11:44 Ra1e8.0/
drwxr-xr-x@ 6818 lmoresi  staff  218176 Aug 29 11:56 Ra1e6.0/

output/cartesian_2x1:
total 0
drwxr-xr-x  6259 lmoresi  staff  200288 Aug 31 10:40 Ra1e6.0/

output/cartesian_1x1:
total 0
drwxr-xr-x@  7022 lmoresi  staff  224704 Sep  6 19:25 Ra1e6.0/
drwxr-xr-x@ 19992 lmoresi  staff  639744 Sep  7 20:29 Ra1e7.0/
drwxr-xr-x@    13 lmoresi  staff     416 Sep 18 11:13 Ra1e8.0/


In [4]:
ls -trl output/cartesian_1x1/Ra1e8.0/_visc0_dt1_delta05/  | tail

-rw-rw-r--@ 1 lmoresi  staff  1079352 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.U.02499.h5
-rw-rw-r--@ 1 lmoresi  staff  1924520 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.T.02499.h5
-rw-rw-r--@ 1 lmoresi  staff   336040 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.P.02499.h5
-rw-rw-r--@ 1 lmoresi  staff     3823 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.02499.xdmf
-rw-rw-r--@ 1 lmoresi  staff   336040 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.eta_n.02500.h5
-rw-rw-r--@ 1 lmoresi  staff  1079352 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.U.02500.h5
-rw-rw-r--@ 1 lmoresi  staff  1924520 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.T.02500.h5
-rw-rw-r--@ 1 lmoresi  staff   336040 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.P.02500.h5
-rw-rw-r--@ 1 lmoresi  staff     3823 Sep 17 20:45 linear_ic_Ra1e8.0_visc0.0_res75.mesh.02500.xdmf
drwxr-xr-x@ 3 lmoresi  staff       96 Sep 18 11:16 _images/


In [5]:
# This is a bit repetitive, sorry 

checkpoint_dir = "output/cartesian_1x1/Ra1e8.0/_visc0_dt1_delta05"
name = "linear_ic_Ra1e8.0_visc0.0"
resolution = 75

checkpoint_base = f"{name}_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"_images", name)
os.makedirs(image_dir, exist_ok=True)

In [6]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CARTESIAN)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
eta_soln = uw.discretisation.MeshVariable("\eta_n", mesh, 1, degree=1)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1, continuous=False)

In [7]:
flux_solver = uw.systems.Projection(mesh, flux)

flux_fn = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_1) + v_soln.sym[1] * t_soln.sym[0]
flux_solver.uw_function = flux_fn
flux_solver.smoothing = 0.0


sample_points = np.zeros((100,2))
sample_points[:,0] = np.linspace(0,1,100)
sample_points[:,1] = 1

sample_points_surf = sample_points.copy()

sample_points[:,1] = 0
sample_points_base = sample_points.copy()



In [8]:
flux_solver.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Projection'>

**Poisson system solver**

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

This solver is formulated in 2 dimensions

In [9]:
steps = range(2000,2000, 25)
calc_flux = False
add_streamlines = False
cmap = "RdBu_r"

In [10]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[750, 750])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        eta_soln.read_timestep(checkpoint_base, "eta_n", step, outputPath=checkpoint_dir)
        if not calc_flux:
            print(f"Plotting {step:05d}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
    pvmesh.point_data["Eta"] = vis.scalar_fn_to_pv_points(pvmesh, eta_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym[0])
    
    # point sources at cell centres
    skip = 15
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)


    pl.add_mesh(
        pvmesh_t,
        cmap=cmap,
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=False,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        clim=[0.0,1.0],
        show_scalar_bar=False
    )


    pl.add_mesh(
        pvmesh,
        cmap="Greys",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=False,
        scalars="Eta",
        use_transparency=False,
        opacity= [0, 0.0, 0.0,  0.5],
        clim=[1.0,10000.0],
        show_scalar_bar=False
    )


    if add_streamlines:
        pvstream = pvmesh.streamlines_from_source(
                point_cloud,
                vectors="V",
                integration_direction="both",
                max_time=1,
                surface_streamlines=True,
            )
    
        pl.add_mesh(pvstream,
                    opacity=0.5,
                    show_scalar_bar=False)

    # pl.add_arrows(pvmesh_v.points, pvmesh_v.point_data["V"], mag=1e-4)

    # pl.camera_position = [(3.0, 0.5, 6.750994846396983),
    #                       (3.0, 0.5, 0.0),
    #                       (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 1000),
                  return_img=False)

    

    if calc_flux:
        flux_solver.solve()
        surf_flux = uw.function.evaluate(flux.sym, sample_points_surf)
        basal_flux = uw.function.evaluate(flux.sym, sample_points_base)
    
        # Heat flux: Upper, Lower, Mean
        print(f"{step:05d},  {surf_flux.mean():8.4f}, {basal_flux.mean():8.4f}, {((surf_flux.mean() + basal_flux.mean())/2):8.4f}")




In [11]:
pl.show(jupyter_backend="html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [12]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [13]:
## t_soln.stats()

In [14]:
I = uw.maths.Integral(mesh, t_soln.sym[0])
mean = I.evaluate()
mean

0.0

In [15]:
I.fn = (t_soln.sym[0]-mean)**2
np.sqrt(I.evaluate())
l2 = np.sqrt(I.evaluate())
l2

0.0

In [16]:
f_steps = range(0,2500, 10)

I = uw.maths.Integral(mesh, 1)
Norm = I.evaluate()

print("Step     Flux(Up)  Flux(Lo)  Flux(Av)  T(Av)    V(rms)")

for step in f_steps:
    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
    except:
        print(f"Failed to read step {step} correctly")
        continue

    flux_solver.solve()
    surf_flux = uw.function.evaluate(flux.sym, sample_points_surf)
    basal_flux = uw.function.evaluate(flux.sym, sample_points_base)

    I.fn = t_soln.sym[0]
    mean_T = I.evaluate() / Norm
    
    I.fn = v_soln.sym.dot(v_soln.sym)
    Vrms = np.sqrt(I.evaluate() / Norm)

    
    # Heat flux: Upper, Lower, Mean
    print(f"{step:05d},  {surf_flux.mean():8.4f}, {basal_flux.mean():8.4f}, {((surf_flux.mean() + basal_flux.mean())/2):8.4f}",
    f"{mean_T:8.4f},  {Vrms:8.4f}")



Step     Flux(Up)  Flux(Lo)  Flux(Av)  T(Av)    V(rms)
00000,    2.1262,   2.2548,   2.1905   0.5000,  955.2940
00010,    1.8351,   2.4196,   2.1274   0.5000,  3747.3509
00020,    1.1121,   2.5266,   1.8194   0.5000,  40588.6577
00030,   -0.8640,   1.6144,   0.3752   0.5000,  87928.7618
00040,   -4.2397,   0.1663,  -2.0367   0.5000,  134994.8293
00050,   -8.9996,  -1.0030,  -5.0013   0.5000,  181508.4739
00060,  -15.1307,  -2.9310,  -9.0309   0.5000,  227564.9106
00070,  -22.6835,  -7.7753, -15.2294   0.5000,  273176.2449
00080,  -31.6003, -15.3971, -23.4987   0.5000,  318184.8171
00090,  -41.2721, -23.3532, -32.3127   0.5000,  361408.1674
00100,  -51.0680, -30.4435, -40.7558   0.5000,  402008.0625
00110,  -60.1195, -36.1558, -48.1377   0.5000,  439611.8617
00120,  -67.2805, -39.6988, -53.4896   0.5000,  473880.8697
00130,  -71.9300, -40.9470, -56.4385   0.4999,  504520.2138
00140,  -74.1367, -40.3478, -57.2423   0.4999,  531282.5069
00150,  -73.8323, -37.9879, -55.9101   0.4999,  5539

In [18]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=1.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="RdYlBu_r",
    edge_color="Black",
    show_edges=False,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

# imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

# pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR

pl.show()


Widget(value='<iframe src="http://localhost:56753/index.html?ui=P_0x3145c78d0_1&reconnect=auto" class="pyvista…